## `train_model.py`

### This file processes the training data and trains the model accordingly

## **Import MediaPipe hands**

In [9]:
import mediapipe as mp

# MediaPipe hands & extract_keypoints from main.py
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=True, 
    max_num_hands=1,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.6
)

I0000 00:00:1738373224.957681 27107037 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M1 Pro


## **Extract keypoints**

#### This `extract_keypoints` function: 
1) Resizes each image
2) Changes image to RGB (Red, Green, Blue) for MediaPipe because OpenCv is in BGR
3) Gets landmarks for each image (INSERT MEDIAPIPE LANDMARK IMAGE HERE)
4) Expected output is a list of 63 values (21 landmarks x 3 coordinates) for each image/hand

In [10]:
import cv2 as cv

def extract_keypoints(image):
    """ Extract hand keypoints from user using MediaPipe """
    try:
        # Revise image -> better preformance
        image = cv.resize(image, (640, 480)) # (640, 480) is default webcam resolution
                                             # Consistent input size will help improve speed 
                                             # and stability of landmark detection

        # Convert image to RGB spectrum
        image_rgb = cv.cvtColor(image, cv.COLOR_BGR2RGB) # OpenCV images are by default BGR format
        results = hands.process(image_rgb) # Process image with MediaPipe
        
        if results.multi_hand_landmarks:
            landmarks = results.multi_hand_landmarks[0].landmark # Get hand's landmark
                                                                 # returns a list of 21 landmark points for a single hand detected
                                                                 # Each landmark has an x, y, and z coordinate
            keypoints = []
            for landmark in landmarks:
                keypoints.append(landmark.x)  # x-coord
                keypoints.append(landmark.y)  # Y-coord
                keypoints.append(landmark.z)  # Z-coord (depth)
            return keypoints
    except Exception as e:
        print(f"Error processing image: {e}")
    return None

    # Expected output is a list of 63 values (21 landmarks x 3 coordinates) for an image/hand

## **Prepare data**

#### This `prepare_data` function:
1) Goes through each class (letter) folder and processes each of the 3000 photos
2) Applies extract_keypoints to each image
3) It then takes each list of 63 values (21 landmarks x 3 coordinates) for each image/hand and saves them as numpy arrays for training

In [11]:
import os
import numpy as np
from sklearn.preprocessing import LabelEncoder

def prepare_data(data_dir, num_samples=None):
    """ Prepare the training data """
    images = []
    labels = []
    sample_count = 0

    # Iterate through each class/letter folder
    for label in os.listdir(data_dir):
        class_folder = os.path.join(data_dir, label)
        if os.path.isdir(class_folder):
            class_images = os.listdir(class_folder)
            print(f"Processing {len(class_images)} images for class {label}")

            # Process each image in the class folder
            for img_name in class_images:
                img_path = os.path.join(class_folder, img_name)
                image = cv.imread(img_path) # Reads the image with OpenCV
                
                if image is not None:
                    keypoints = extract_keypoints(image)  # MediaPipe landmarks applied to image if hand detected 
                    if keypoints:
                        images.append(keypoints) 
                        labels.append(label)
                        sample_count += 1
                        
                        # Optionally stop after 'num_samples' images for quick testing
                        if num_samples and sample_count >= num_samples:
                            break

    # Convert lists to numpy arrays
    images = np.array(images)
    labels = np.array(labels)

    print(f"Total images loaded: {len(images)}")

    # Encode labels as integers
    label_encoder = LabelEncoder() # sklearn
    labels = label_encoder.fit_transform(labels)
    
    return images, labels, label_encoder

W0000 00:00:1738373224.983266 27143233 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


## **Prepare the data**

#### Load the dataset, saving X, y, label_encoder for each image after processed

In [12]:
# Prepare the data
data_dir = '/Users/carlopisacane/Desktop/Honors/HonorsThesis/data/asl_alphabet_train'  # Path to your dataset
X, y, label_encoder = prepare_data(data_dir)

W0000 00:00:1738373225.041209 27143233 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Processing 3000 images for class R
Processing 3000 images for class U
Processing 3000 images for class I
Processing 3000 images for class N
Processing 3000 images for class G
Processing 3000 images for class Z
Processing 3000 images for class T
Processing 3000 images for class S
Processing 3000 images for class A
Processing 3000 images for class F
Processing 3000 images for class O
Processing 3000 images for class H
Processing 3000 images for class nothing
Processing 3000 images for class M
Processing 3000 images for class J
Processing 3000 images for class C
Processing 3000 images for class D
Processing 3000 images for class V
Processing 3000 images for class Q
Processing 3000 images for class X
Processing 3000 images for class E
Processing 3000 images for class B
Processing 3000 images for class K
Processing 3000 images for class L
Processing 3000 images for class Y
Processing 3000 images for class P
Processing 3000 images for class W
Total images loaded: 56270


## **Train the model**

#### This model is trained by:
1) Split the data into training and validation sets
2) Build a machine learning model with two **Dense** layers and **Dropout** for regularization
3) Train the model on the extracted keypoints for 10 epochs
4) Save the trained model to a file named `asl_model.h5`

### Split data into training and validation sets

In [14]:
from sklearn.model_selection import train_test_split

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
# Training set (X_train, y_train)
# Validation set (X_val, y_val)  
print("Training samples:", X_train.shape[0])
print("Validation samples:", X_val.shape[0])

# test_size=0.2 means 20% of data goes to validation and 80% remains for training model
# random_state=42 ensures that the split is reproducible

# You need validation because the model is always adjusting weights to minimize loss,
# you risk overfitting the model if only track performance on data being trained.

# In model.fit -> the validation data is set at the end of each epoch and reports the validation
# loss and accuracy. 

Training samples: 45016
Validation samples: 11254


### Build and compile the model

In [15]:
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Dropout#, Flaten

# Build the model with regularization
model = Sequential([
    Dense(128, input_dim=X_train.shape[1], activation='relu'), # Input layer and first dense layer
    Dropout(0.5), # Sets 50% of inputs to zero, help prevent overfitting
    Dense(64, activation='relu'), # Second dense layer
    Dropout(0.5), # Another dropout layer
    Dense(26, activation='softmax') # Softmax outputs probability distribution across all 26 classes
])

# Compile the model
model.compile(
    optimizer='adam', # Adapts the learning rate for each parameter using Adam
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy']
)

model.summary() # Print model architecture summary

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 128)            │         8,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 26)             │         1,690 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 18,138 (70.85 KB)

 Trainable params: 18,138 (70.85 KB)

 Non-trainable params: 0 (0.00 B)

### Train the model and save it in `asl_model.h5`

In [18]:
# Train the model
model.fit(
    X_train, y_train, 
    epochs=10, 
    validation_data=(X_val, y_val)
)

model.save('model/asl_model.h5')  # Save the trained model
print("Model saved as asl_model.h5")

Epoch 1/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 1s 890us/step - accuracy: 0.9034 - loss: 0.2901 - val_accuracy: 0.9594 - val_loss: 0.1340
Epoch 2/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 1s 827us/step - accuracy: 0.9012 - loss: 0.2996 - val_accuracy: 0.9632 - val_loss: 0.1171
Epoch 3/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 1s 814us/step - accuracy: 0.9057 - loss: 0.2897 - val_accuracy: 0.9662 - val_loss: 0.1178
Epoch 4/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 1s 816us/step - accuracy: 0.9098 - loss: 0.2847 - val_accuracy: 0.9684 - val_loss: 0.1124
Epoch 5/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 1s 740us/step - accuracy: 0.9051 - loss: 0.2912 - val_accuracy: 0.9687 - val_loss: 0.1147
Epoch 6/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 1s 750us/step - accuracy: 0.9091 - loss: 0.2767 - val_accuracy: 0.9618 - val_loss: 0.1107
Epoch 7/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 1s 735us/step - accuracy: 0.9081 - loss: 0.2740 - val_accuracy: 0.9653 - val_loss: 0.1228
Epoch 8/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 1s 794us/step - accuracy: 0.9051 -

Model saved as asl_model.h5
